# Orbit

In [ ]:
from manim import *
# conda activate my-manim-environment
import numpy as np

### Orbit

In [ ]:
class OrbitingPlanet(Scene):
    def slide_rotated_ellipse(self):
        self.make_elliptical_orbit()
        self.w_offset = np.pi
        self.rotate_angle = 0

        eq_linear_tra = MathTex(r't_\text{I}', '=', 't_0', '+', 'P', 'E') \
            .move_to(1.5 * RIGHT + 2*UP)
        eq_eccentric1 = MathTex(r't_\text{II} \simeq t_\text{I} + \frac{P}{2} + ',
                                r'\frac{2P}{\pi}','e').move_to(2.1 * RIGHT + 1*UP)
        eq_eccentric2 = MathTex(r't_\text{II} \simeq t_\text{I} + \frac{P}{2} + '
                                r'\frac{2P}{\pi}','e\cos{\omega}').move_to(2.7 * RIGHT + 1*UP)
        v1 = MathTex(r'\small{e', r'\rightarrow \text{orbital eccentricity}}')\
            .move_to(1*DOWN+3*RIGHT)
        v2 = MathTex(r'\small{\omega', r'\rightarrow \text{argument of periapsis}}')\
            .move_to(2*DOWN+3.5*RIGHT)
        vbox1 = SurroundingRectangle(eq_eccentric1[2], buff=.1)
        vbox2 = SurroundingRectangle(eq_eccentric2[1], buff=.1)

        def update_planet_rotated(mobject, dt):
            theta = self.w_offset + self.v * dt
            theta = self.wrap(theta)
            r = (self.semimajor * (1 - self.e**2)) / (1 + self.e * np.cos(theta))
            x = r * np.cos(theta)
            y = r * np.sin(theta)

            mobject.move_to(np.array([-x, -y, 0])
                            + self.star_elliptical.get_center() + self.semimajor*LEFT)
            mobject.rotate(angle=self.rotate_angle,about_point=self.star_elliptical.get_center())
            self.w_offset = theta

        title_ellipse = Tex("Eccentric Orbit").to_corner(UP + LEFT)
        self.add(title_ellipse, self.orbit2.shift(2.5*LEFT), eq_linear_tra)
        self.elliptical_orbit.shift(2.5*LEFT)

        transit_square = Square(side_length=0.7, color=PURE_RED) \
                                    .move_to(self.star_elliptical.get_center() + 1.5*DOWN)
        eclipse_square = Square(side_length=0.7, color=PURE_RED) \
                                    .move_to(self.star_elliptical.get_center() + 1.5*UP)
        text_transit = Tex(r'\tiny{Transit}')\
            .move_to(transit_square.get_center() + 1*RIGHT + 0.25*DOWN)
        text_eclipse = Tex(r'\tiny{Eclipse}')\
            .move_to(eclipse_square.get_center() + 1*RIGHT + 0.25*UP)
        all = VGroup(transit_square, eclipse_square, text_transit, text_eclipse)

        self.planet_elliptical.add_updater(update_planet_rotated)
        self.wait(1.5*self.period)
        self.add(text_transit, transit_square)
        self.wait(1.5 * self.period)
        self.add(text_eclipse, eclipse_square)
        self.add(eq_eccentric1)
        self.play(Create(vbox1), Write(v1))
        self.wait(2 * self.period)
        self.planet_elliptical.remove_updater(update_planet_rotated)

        self.rotate_angle = np.pi/4
        self.play(Rotate(self.orbit2,angle=self.rotate_angle,
                         about_point=self.star_elliptical.get_center()),
                  Transform(eq_eccentric1,eq_eccentric2),
                  transit_square.animate.shift(0.75*DOWN), eclipse_square.animate.shift(0.4*DOWN),
                  FadeOut(text_eclipse), FadeOut(text_transit),
                  ReplacementTransform(vbox1, vbox2))
        self.add(v2)
        self.planet_elliptical.add_updater(update_planet_rotated)
        self.wait(2*self.period)
        self.planet_elliptical.remove_updater(update_planet_rotated)
        self.wait()

    def wrap(self,angle):
        if angle >= 2 * np.pi: 
            return angle - 2 * np.pi
        elif angle <= 0:
            return angle + 2 * np.pi
        else:
            return angle
    def make_circular_orbit(self):
        # make dots for the star and planet
        self.star = Dot(color=YELLOW).scale(5)
        self.planet = Dot(color=DARK_BROWN).scale(2)
        # make a circle for the orbit
        self.circular_orbit = Circle(radius=2, color=GREY_A, stroke_width=1)
        # shift the circular orbit to be centered on the star
        self.circular_orbit.move_to(self.star.get_center())
        # shift the planet to be on the orbit at an angle of 0 degrees
        self.planet.move_to(self.circular_orbit.point_at_angle(0 * DEGREES))
        # Todo: why dashed circle?
        self.dashed_circle = DashedVMobject(self.circular_orbit, num_dashes=40)
        # group all of the objects into a single attribute 
        self.orbit_C = VGroup(self.star, self.dashed_circle, self.planet)
        
    def make_elliptical_orbit(self):
        # Define ellipse parameters
        self.ecc = 0.5          # eccentricity
        self.a = 2      # semimajor axis
        self.b = self.semimajor * np.sqrt(1 - self.ecc ** 2)  # Semi-minor axis
        # calculate the orbital period and angular velocity
        self.period = np.sqrt(self.a ** 3)  # Period
        self.v = 2 * PI / self.period           # angular velocity

        self.star_E = Dot(color=YELLOW).scale(5)
        self.planet_E = Dot(color=DARK_BROWN).scale(2)
        self.elliptical_orbit = Ellipse(width=2 * self.a, height=2 * self.b, color=GREY_A, stroke_width=1)
        self.elliptical_orbit.move_to(self.star_elliptical.get_center() + (self.semimajor * self.e) * LEFT)
        self.planet_E.move_to(self.elliptical_orbit.point_at_angle(0 * DEGREES))
        self.dashed_ellipse = DashedVMobject(self.elliptical_orbit, num_dashes=40)
        self.orbit_E = VGroup(self.star_E, self.dashed_ellipse, self.planet_E)
        
        
    